In [1]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
%matplotlib inline

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 600})
plt.rcParams['lines.linewidth'] = 0.5 

In [113]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-8h.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()
odf

,timestamp,open,high,low,close,volume,date,idate
date,,,,,,,,
2021-01-01 00:00:00,1609459200000,28923.63,29470.00,28690.17,29092.83,18869.366827,2021-01-01 00:00:00,20210101
2021-01-01 08:00:00,1609488000000,29092.84,29600.00,28872.24,29188.67,20078.654834,2021-01-01 08:00:00,20210101
2021-01-01 16:00:00,1609516800000,29188.67,29360.00,28624.57,29331.69,15234.903350,2021-01-01 16:00:00,20210101
2021-01-02 00:00:00,1609545600000,29331.70,29820.50,28946.53,29750.00,17258.671371,2021-01-02 00:00:00,20210102
2021-01-02 08:00:00,1609574400000,29751.47,31800.00,29473.91,31691.29,43255.778007,2021-01-02 08:00:00,20210102
...,...,...,...,...,...,...,...,...
2023-05-11 00:00:00,1683763200000,27598.74,27622.68,27367.76,27450.07,9819.590800,2023-05-11 00:00:00,20230511
2023-05-11 08:00:00,1683792000000,27450.07,27630.14,26952.00,27179.42,19518.716970,2023-05-11 08:00:00,20230511
2023-05-11 16:00:00,1683820800000,27179.43,27219.52,26702.05,26968.62,18297.005880,2023-05-11 16:00:00,20230511
